In [1]:
import sympy as sp

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)

orth = (A.T @ A - sp.eye(2)).norm(ord='fro') ** 2

avec = sp.Matrix([a11, a12, a21, a22])



In [ ]:
print(gen_rust("orth", orth))

In [2]:
grad = sp.Matrix([orth.diff(x) for x in avec])
grad

Matrix([
[4*a11*(a11**2 + a21**2 - 1) + 4*a12*(a11*a12 + a21*a22)],
[4*a11*(a11*a12 + a21*a22) + 4*a12*(a12**2 + a22**2 - 1)],
[4*a21*(a11**2 + a21**2 - 1) + 4*a22*(a11*a12 + a21*a22)],
[4*a21*(a11*a12 + a21*a22) + 4*a22*(a12**2 + a22**2 - 1)]])

In [3]:
hess = sp.Matrix(
    [
        [grad[i].diff(avec[j]) for j in range(4)]
        for i in range(4)
    ]
)
hess

Matrix([
[12*a11**2 + 4*a12**2 + 4*a21**2 - 4,               8*a11*a12 + 4*a21*a22,               8*a11*a21 + 4*a12*a22,                           4*a12*a21],
[              8*a11*a12 + 4*a21*a22, 4*a11**2 + 12*a12**2 + 4*a22**2 - 4,                           4*a11*a22,               4*a11*a21 + 8*a12*a22],
[              8*a11*a21 + 4*a12*a22,                           4*a11*a22, 4*a11**2 + 12*a21**2 + 4*a22**2 - 4,               4*a11*a12 + 8*a21*a22],
[                          4*a12*a21,               4*a11*a21 + 8*a12*a22,               4*a11*a12 + 8*a21*a22, 4*a12**2 + 4*a21**2 + 12*a22**2 - 4]])

In [ ]:
import sympy as sp
from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
        prefix="rust_code"
    )[0][1]

In [ ]:
for i, g in enumerate(grad):
    print(gen_rust(f"grad_{i}", g))
    print()

In [ ]:
for i in range(hess.shape[0]):
    for j in range(hess.shape[1]):
        h = hess[i, j]
        print(gen_rust(f"hess_{i}_{j}", g))
        print()

In [ ]:
hess